In [ ]:
%reload_kedro

In [ ]:
import pymongo
import pandas as pd
import os
import json

In [ ]:
catalog.list()

In [ ]:
DATABASE_NAME = 'ogs'
GEO_COMPONENTS_COLLECTION_NAME = 'geo_components'
DATA_SOURCES_COLLECTION_NAME = 'data_sources'

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.get_database(DATABASE_NAME)

In [ ]:
def cast_json(line):
    try:
        json_sample = json.loads(line)
    except:
        print('failed to parse to json:', line)
        return {}
    return json_sample

# geo_components

Delete existing geo_components collection

In [ ]:
db.drop_collection(GEO_COMPONENTS_COLLECTION_NAME)
db.create_collection(GEO_COMPONENTS_COLLECTION_NAME)
geo_components_collection = db[GEO_COMPONENTS_COLLECTION_NAME]

In [ ]:
## Create some geo_components from GCP data
def get_sample_countries_geo_components():
    # Read gcp file
    with open('./../data/01_raw/gcp-carbon-v2.json', 'r') as f:
        lines = f.readlines()
        
    # Transform to json
    jsons = [
        cast_json(line) for line in lines
    ]
    
    # Extract geo components
    geo_components = [
        json_sample['geo_component'] for json_sample in jsons
    ]
    
    # Distinct dicts
    countries = list({v['identifier']['id']:v for v in geo_components}.values())
    
    # Format
    ref_countries = [
        {
            'type': 'Country',
            'identifiers': {
                'alpha-3': country['identifier']['id']
            }
        }
        for country in countries
    ]
       
    return ref_countries

In [ ]:
countries = get_sample_countries_geo_components()

geo_components_collection.insert_many(countries)

# data_sources

In [ ]:
db.drop_collection(DATA_SOURCES_COLLECTION_NAME)
db.create_collection(DATA_SOURCES_COLLECTION_NAME)
data_sources_collection = db[DATA_SOURCES_COLLECTION_NAME]

In [ ]:
data_sources = catalog.load('data_sources')

data_sources = data_sources.split('\n')

data_sources = [
    cast_json(data_source) for data_source in data_sources
]

In [ ]:
data_sources

In [ ]:
data_sources_collection.insert_many(data_sources)

# Exploration

In [ ]:
list(geo_components_collection.find({}))

In [ ]:
list(data_sources_collection.find({}))